## Creaton of Process Flow Diagram Pull-down Cloze Questions for Moodle Quiz
### Copy and paste output field into the Moodle Quiz Input Field
### Use [this Google Sheet](https://docs.google.com/spreadsheets/d/1els95ulHaqDJ2tmeWbkxdvpmwBapMH6CZXIHPbQu_-s/edit#gid=0) to create CSVs for input to the generator (in Google Sheets: file -> download -> CSV)

## Press CTRL + F9 to run all

wish list 


*   htlm new line DONE
*   Longer partial credit response DONE
*   More than 10 steps -30 -- I DON'T think this is working yet. It gives me an error everytime I put more than 10. DONE??
*   Allow multiple uploads or delete old file after use (PFD.csv) DONE
*   Each step on a new line, not yet working DONE
*   Introduce different mark weight per step DONE
*   Ask if a new list of inputs is needed or not





In [1]:
import numpy as np
import pandas as pd
import os, sys
# import glob
from google.colab import files

blank = 'n/a'

Take the lists of possible values for processes and modifiers. 
## ONLY RUN THIS CELL IF A NEW LIST IS NEEDED


In [8]:
#try:
#  OptionsFilename
#except NameError:

print('Upload the CSV file with possible processes & modifiers')
fileOptions = files.upload()
OptionsFilename = next(iter(fileOptions))
os.rename(OptionsFilename, 'PFDInputs.csv')

#read PFD response options
PFDinputs = pd.read_csv('PFDInputs.csv')
PFDinputs

#  truncate and add n/a values to process and modifier options
Processes = PFDinputs["Process Options"]
Processes = (Processes.dropna()).values.tolist()

Processes.append(blank)

print(type(Processes))
print(Processes)

Modifiers = PFDinputs["Process Modifiers"]
Modifiers = (Modifiers.dropna()).values.tolist()
#Modifiers = Modifiers.append(pd.Series([blank]), ignore_index = True)
Modifiers.append(blank)
print(Modifiers)

Upload the CSV file with possible processes & modifiers


Saving Cloze Generator - PFD Inputs.csv to Cloze Generator - PFD Inputs.csv
<class 'list'>
['UV Lithography', 'DUV Lithography', 'EUV Lithography', 'DWL', 'Electro beam lithography (EBL)', 'Thermal Oxidation', 'Thermal Evaporation (ebeam)', 'Sputtering', 'CVD', 'LPCVD', 'PECVD', 'Lift-Off', 'Vapor etching', 'Pure chemical plasma etching', 'Pure physical plasma etching', 'RIE', 'DRIE', 'Inhibitor driven anisotropy', 'Standard wet etching (isotropic)', 'Anisotropic wet etching (KOH)', 'Electro deposition', 'n/a']
['Silicon (Si)', 'poly Silicon (p-Si)', 'SiO<sub>2</sub> (Silicon Dioxide)', 'Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride)', 'Al (Aluminium)', 'Au (Gold)', 'Cr (Chromium)', 'Ni (Nickel)', 'Pt (Platinum)', 'Ti (Titanium)', 'n/a']


Take the Process Flow Diagram Marking Scheme

In [10]:
print('Upload the CSV file with the diagram question parameters')
fileDiagram = files.upload()
#print("the file name is: " + str(fileDiagram))
DiagramFilename = next(iter(fileDiagram))
os.rename(DiagramFilename, 'PFD.csv')

Upload the CSV file with the diagram question parameters


Saving Cloze Generator - PFD Generator.csv to Cloze Generator - PFD Generator.csv


Create dataframes from csv inputs

In [11]:
#read PFD schematic
df = pd.read_csv('PFD.csv', header=None, usecols=[0,1,2,3,4,5,6,7,8,9,10], skiprows=5,nrows=30)
#print('Upload the CSV file with the diagram question parameters')
##### df = pd.read_csv(files.upload(), header=None, usecols=[0,1,2,3,4,5,6,7,8], skiprows=5,nrows=30)
#check for longest of columns 0, 1, 3, or 6 and trim to length of longest
trim = int(0)
for col in [0,1,3,6]:
  key = df.index.get_loc(df[col].last_valid_index())
  if key > trim:
    trim = key
  else:
    break

df = df.truncate(after = trim)


# drop column 0
df = df.drop([0], axis=1)


print(df)
df.columns = ["Pro_full","Grade","Mod_full","Grade_mod","Pro_part_1","Mod_part_1","Part_1","Pro_part_2","Mod_part_2","Part_2"]
df['Grade'] = (df['Grade'].fillna(1)).astype(int)
df['Grade_mod'] = (df['Grade_mod'].fillna(1)).astype(int)
df['Part_1'] = (df['Part_1'].fillna(0)).astype(int)
df['Part_2'] = (df['Part_2'].fillna(0)).astype(int)
df = df.replace(np.nan, blank, regex=True)

df
#delete CSV
os.remove('PFD.csv')

df

                            1   2   ...  9   10
0                        PECVD   2  ... NaN NaN
1               UV Lithography   2  ... NaN NaN
2  Thermal Evaporation (ebeam)   2  ... NaN NaN
3                     Lift-Off   2  ... NaN NaN
4                        PECVD   2  ... NaN NaN
5               UV Lithography   2  ... NaN NaN
6                          RIE   2  ... NaN NaN
7                          NaN   1  ... NaN NaN
8                          NaN   1  ... NaN NaN
9                          NaN   1  ... NaN NaN

[10 rows x 10 columns]


,Pro_full,Grade,Mod_full,Grade_mod,Pro_part_1,Mod_part_1,Part_1,Pro_part_2,Mod_part_2,Part_2
0,PECVD,2,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),1,Sputtering,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),50,n/a,n/a,0
1,UV Lithography,2,n/a,1,n/a,n/a,0,n/a,n/a,0
2,Thermal Evaporation (ebeam),2,Al (Aluminium),1,Sputtering,Al (Aluminium),25,n/a,n/a,0
3,Lift-Off,2,n/a,1,n/a,n/a,0,n/a,n/a,0
4,PECVD,2,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),1,Sputtering,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),50,n/a,n/a,0
5,UV Lithography,2,n/a,1,n/a,n/a,0,n/a,n/a,0
6,RIE,2,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),3,n/a,n/a,0,n/a,n/a,0
7,n/a,1,n/a,1,n/a,n/a,0,n/a,n/a,0
8,n/a,1,n/a,1,n/a,n/a,0,n/a,n/a,0
9,n/a,1,n/a,1,n/a,n/a,0,n/a,n/a,0


Confirm size of question

In [12]:
# Check if there are any blanks in the full-valule solution
if df.Pro_full[len(df.Pro_full)-1] != blank:
  firstBlank = len(df.Pro_full)-1
else:
    firstBlank = (df[df.Pro_full == blank].index.values)[0] #this was originally looking for n/a's, but now has to look for blanks
#print(type(firstBlank))
print('There are ' + str(firstBlank+1) + ' steps in the full-credit solution, and ' + str(len(df)) + ' rows provided.\n')
answer = input("Would you like to limit the number of rows\n>(y/N)?")
#print(answer), print(type(answer))
if answer == '' : answer = 'n'

if answer[0] == "y" or answer[0] == "Y":
  #print('answer is yes')
  length = input("Provide the number of rows (<" + str(len(df)) +")\n")
  if length == '' : length = str(len(df))
  newLimit = int(length)
  #print(type(newLimit))
  if newLimit < firstBlank:
    newLimit = firstBlank 
    print('Cannot make the question shorter than the primary process (' + str(newLimit) + ')')
  elif newLimit > len(df):
    newLimit = len(df) 
    print('No change, question lengh remains at ' + str(newLimit))
  else:
    print('New question length = ' + str(newLimit))
    
else:
  newLimit = len(df) 
#print(newLimit)
df = df.truncate(after = (newLimit-1))
#dfNew
#df[Pro_full]
#df = df.apply(int)
df



There are 8 steps in the full-credit solution, and 10 rows provided.

Would you like to limit the number of rows
>(y/N)?n


,Pro_full,Grade,Mod_full,Grade_mod,Pro_part_1,Mod_part_1,Part_1,Pro_part_2,Mod_part_2,Part_2
0,PECVD,2,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),1,Sputtering,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),50,n/a,n/a,0
1,UV Lithography,2,n/a,1,n/a,n/a,0,n/a,n/a,0
2,Thermal Evaporation (ebeam),2,Al (Aluminium),1,Sputtering,Al (Aluminium),25,n/a,n/a,0
3,Lift-Off,2,n/a,1,n/a,n/a,0,n/a,n/a,0
4,PECVD,2,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),1,Sputtering,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),50,n/a,n/a,0
5,UV Lithography,2,n/a,1,n/a,n/a,0,n/a,n/a,0
6,RIE,2,Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride),3,n/a,n/a,0,n/a,n/a,0
7,n/a,1,n/a,1,n/a,n/a,0,n/a,n/a,0
8,n/a,1,n/a,1,n/a,n/a,0,n/a,n/a,0
9,n/a,1,n/a,1,n/a,n/a,0,n/a,n/a,0


# Create the output to copy/paste into the Moodle Quiz (Cloze type question)

In [13]:
output = ""
for grade,step,grade_mod in zip(df["Grade"],list(range(0,len(df["Pro_full"]))),df["Grade_mod"]):
#for step in list(range(0,len(df["Pro_full"]))):
    output += "Step "
    if len(str(step+1)) == 1: output += '  '
    output += str(step+1)

# this line to be removed ->
#    output += ": {2:MULTICHOICE:"

# <<< Uncomment the below when ready and remove above >>>
    output += ": {"
# Concatenate step value (Grade)
    output += str(grade)

    output += ":MULTICHOICE:"

    for i in Processes:
# set value awarded for process (Full, Partial 1, or Partial 2)
        output += "%"
        if df["Pro_full"][step] == i:
            output += "100"
        elif df["Pro_part_1"][step] == i: #this is where the problem is
            output += str(df["Part_1"][step])
        elif df["Pro_part_2"][step] == i:
            output += str(df["Part_2"][step])
        else:
            output += "0"
        output += "%"
        output += i
        if i != Processes[-1]:
            output += "~"
        else:
            break

# set value awarded for modifier (Full, Partial 1, or Partial 2)
    output += "}"

    output += " of {"
    output += str(grade_mod)
    output += ":MULTICHOICE:"
    for i in Modifiers:
        output += "%"
        if df["Mod_full"][step] == i:
            output += "100"
        elif df["Mod_part_1"][step] == i:  #this is where the problem is
            output += str(df["Part_1"][step])
        elif df["Mod_part_2"][step] == i:
            output += str(df["Part_2"][step])
        else:
            output += "0"
        output += "%"
        # currently inputting n/a%n/a rather than 0%n/a
        output += i
        if i != Modifiers[-1]:
            output += "~"
        else:
            break
    output += "}"
#    output += '<br>'
    output += "\n\r\n"
print(output)

Step   1: {2:MULTICHOICE:%0%UV Lithography~%0%DUV Lithography~%0%EUV Lithography~%0%DWL~%0%Electro beam lithography (EBL)~%0%Thermal Oxidation~%0%Thermal Evaporation (ebeam)~%50%Sputtering~%0%CVD~%0%LPCVD~%100%PECVD~%0%Lift-Off~%0%Vapor etching~%0%Pure chemical plasma etching~%0%Pure physical plasma etching~%0%RIE~%0%DRIE~%0%Inhibitor driven anisotropy~%0%Standard wet etching (isotropic)~%0%Anisotropic wet etching (KOH)~%0%Electro deposition~%0%n/a} of {1:MULTICHOICE:%0%Silicon (Si)~%0%poly Silicon (p-Si)~%0%SiO<sub>2</sub> (Silicon Dioxide)~%100%Si<sub>3</sub>N<sub>4</sub> (Silicon Nitride)~%0%Al (Aluminium)~%0%Au (Gold)~%0%Cr (Chromium)~%0%Ni (Nickel)~%0%Pt (Platinum)~%0%Ti (Titanium)~%0%n/a}

Step   2: {2:MULTICHOICE:%100%UV Lithography~%0%DUV Lithography~%0%EUV Lithography~%0%DWL~%0%Electro beam lithography (EBL)~%0%Thermal Oxidation~%0%Thermal Evaporation (ebeam)~%0%Sputtering~%0%CVD~%0%LPCVD~%0%PECVD~%0%Lift-Off~%0%Vapor etching~%0%Pure chemical plasma etching~%0%Pure physical pl